In [1]:
import numpy as np
import cv2

In [2]:
OPEN_HAND = './Pictures/openHand.png'
ROCK_HAND = './Pictures/rockHand.png'
SCISSOR_HAND = './Pictures/scissorHand.png'
START_HAND = './Pictures/startHand.png'

In [3]:
def threshold(frame):
    # Skin detection and creating binary image based on detected skin
    w,h = frame.shape[0:2]
    img_thres = np.zeros((w,h), dtype="uint8")
    mask = np.logical_and.reduce([frame[:,:,0] >= 50, frame[:,:,1] >= 95, frame[:,:,2] >= 175])
    img_thres[mask] = 255
    
    return img_thres

def find_contour(img_thres):
    #Finds contours from a thresholded image. Returns the coordinates for drawing a rectangle.
    contours, hierarchy = cv2.findContours(img_thres, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_output = cv2.cvtColor(np.zeros(np.shape(img_thres), dtype="uint8"), cv2.COLOR_GRAY2BGR)
    
    bound_rect = None
    
    if(len(contours) > 0):
        max_id = max(enumerate(contours), key=lambda x: cv2.contourArea(x[1]))[0]
        max_size = cv2.contourArea(contours[max_id])
        bound_rect = cv2.boundingRect(contours[max_id])
        hull = [cv2.convexHull(contours[max_id])]
        cv2.drawContours(contour_output, hull, -1, (255,0,0), 1, 8)
        
        contour_output = roi.copy()
        
        cv2.drawContours(contour_output, contours, max_id, (255,0,0), cv2.FILLED, 8)
        cv2.drawContours(contour_output, contours, max_id, (0,0,255), 2, 8)
        cv2.rectangle(contour_output, bound_rect, (0,255,0), 1,8,0)
    cv2.imshow("Contour Output", contour_output)
    return bound_rect

In [4]:
def detect_gesture(frame, cropped, method=cv2.TM_CCOEFF_NORMED):
    # Detects the hand gestures through the Normalized Correlation Coefficient method.
    open_hand = cv2.imread(OPEN_HAND)
    scissor_hand = cv2.imread(SCISSOR_HAND)
    rock_hand = cv2.imread(ROCK_HAND)
    start_hand = cv2.imread(START_HAND)
    
    open_hand = cv2.cvtColor(open_hand, cv2.COLOR_BGR2GRAY)
    scissor_hand = cv2.cvtColor(scissor_hand, cv2.COLOR_BGR2GRAY)
    rock_hand = cv2.cvtColor(rock_hand, cv2.COLOR_BGR2GRAY)
    start_hand = cv2.cvtColor(start_hand, cv2.COLOR_BGR2GRAY)
    
    _,open_hand = cv2.threshold(open_hand, 180, 255,0)
    _,scissor_hand = cv2.threshold(scissor_hand, 180, 255, 0)
    _,rock_hand = cv2.threshold(rock_hand, 180,  255,0)
    _,start_hand = cv2.threshold(start_hand, 180,  255,0)
    
    max_val_dict = {'open': None, 'rock': None, 'scissor': None, 'start': None}
    
    try:
        res_open = cv2.matchTemplate(cropped,open_hand,method)
        res_rock = cv2.matchTemplate(cropped,rock_hand,method)
        res_scissor = cv2.matchTemplate(cropped,scissor_hand,method)
        res_start = cv2.matchTemplate(cropped,start_hand,method)
        
        _,max_val_open, _, _ = cv2.minMaxLoc(res_open)
        _,max_val_rock, _, _ = cv2.minMaxLoc(res_rock)
        _,max_val_scissor, _, _ = cv2.minMaxLoc(res_scissor)
        _,max_val_start, _, _ = cv2.minMaxLoc(res_start)
        
        max_val_dict['open'] = max_val_open
        max_val_dict['rock'] = max_val_rock
        max_val_dict['scissor'] = max_val_scissor
        max_val_dict['start'] = max_val_start
       
        maximum = max(max_val_dict.values(), key=float)      
        
        if maximum == max_val_dict['open']:
            gesture = 'Open Hand'
        elif maximum == max_val_dict['rock']:
            gesture ='rock Hand'
        elif maximum == max_val_dict['scissor']:
            gesture ='scissor Hand'
        elif maximum == max_val_dict['start']:
            gesture = 'start'
        return gesture
    except Exception as e:
        print(e)
        pass

In [5]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    roi = frame[100:265, 100:265] # Crop image
    cv2.rectangle(frame, (100,100), (265,265), (0,255,0)) #draw a rectangle in frame for the region of interest
    
    roi = cv2.GaussianBlur(roi, (5,5), 0) # remove excess noise by blurring
    roi = cv2.dilate(roi, (2,2)) # Then dialating the image
        
    img_thres = threshold(roi)
    bound_rect = find_contour(img_thres)
    gesture = detect_gesture(frame, img_thres)
    frame = cv2.putText(frame, gesture, (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))
    cv2.imshow('frame',frame)
    cv2.imshow('ROI', img_thres)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()